# Standard Regression (BQML)

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

[https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

This solution, including any related sample code or data, is made available 
on an “as is,” “as available,” and “with all faults” basis, solely for 
illustrative purposes, and without warranty or representation of any kind. 
This solution is experimental, unsupported and provided solely for your 
convenience. Your use of it is subject to your agreements with Google, as 
applicable, and may constitute a beta feature as defined under those 
agreements.  To the extent that you make any data available to Google in 
connection with your use of the solution, you represent and warrant that you 
have all necessary and appropriate rights, consents and permissions to permit 
Google to use and process that data.  By using any portion of this solution, 
you acknowledge, assume and accept all risks, known and unknown, associated 
with its usage, including with respect to your deployment of any portion of 
this solution in your systems, or usage in connection with your business, 
if at all.

## 0) Dependencies

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = '' #add proj name and dataset

In [ ]:
# Google credentials authentication libraries
from google.colab import auth

!pip install --upgrade -q gspread 
import gspread

from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())


# BigQuery Magics
'''
BigQuery magics are used to run BigQuery SQL queries in a python environment.
These queries can also be run in the BigQuery UI
'''

from google.cloud import bigquery
from google.cloud.bigquery import magics
magics.context.project = project_name #update project name 
client = bigquery.Client(project=magics.context.project)
%load_ext google.cloud.bigquery
bigquery.USE_LEGACY_SQL = False


# data processing libraries
import numpy as np
import pandas as pd


# modeling and metrics
from statsmodels.stats.stattools import durbin_watson
import statsmodels.api as sm

!pip install relativeImp
from relativeImp import relativeImp


# visutalization
import matplotlib.pyplot as plt
import seaborn as sns

## 1) Import dataset

In [ ]:
'''
Import the data using the bigquery magics (%% command). 
Pulls all of the data from the cleaned data table and stores into a dataframe "df"
'''

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = '' #add proj name and dataset

In [ ]:
%%bigquery df
SELECT *
FROM `.RBA_demo.cleaned_data`; #update project name

In [ ]:
df.columns

Index(['x8', 'x9', 'x10', 'x11', 'x12', 'x14', 'x16', 'x26', 'x27', 'x28',
       'x29', 'x30', 'x32', 'x33', 'x34', 'x35', 'x37', 'x38', 'x39', 'x40',
       'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'y1'],
      dtype='object')

In [ ]:
df.describe()

,x8,x9,x10,x11,x12,x14,x16,x26,x27,x28,x29,x30,x32,x33,x34,x35,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,y1
count,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,6.070000e+02,607.000000
mean,6.771080e-16,1.365739e-15,-1.236975e-15,-1.832691e-16,-5.739506e-16,-7.813629e-16,1.123026e-16,9.873120e-16,-1.616865e-16,6.972274e-16,1.653446e-16,-2.407007e-16,-9.145165e-19,-5.607815e-16,-5.150557e-16,-2.225933e-16,9.030850e-17,-3.777868e-16,-6.333941e-16,-4.298228e-16,-5.183479e-16,1.015571e-16,6.543366e-16,2.864266e-16,1.810743e-16,-7.133229e-17,2250.153213
std,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,1.000825e+00,461.357669
min,-1.733117e+00,-2.498340e+00,-3.140406e+00,-1.890599e+00,-2.525121e+00,-2.705138e+00,-2.254904e+00,-9.612590e-01,-1.594503e+00,-1.051488e+00,-1.623625e+00,-1.880844e+00,-3.541561e+00,-2.930653e+00,-1.760821e+00,-2.360258e+00,-4.111161e+00,-1.685367e+00,-3.071005e+00,-5.696982e+00,-2.518715e+00,-2.806060e+00,-1.232860e+00,-2.583499e+00,-2.810452e+00,-2.421011e+00,1159.000000
25%,-7.996660e-01,-7.817463e-01,-7.018516e-01,-8.989277e-01,-7.769680e-01,-6.845193e-01,-1.129063e-02,-9.612590e-01,-3.782040e-01,-1.051488e+00,-8.304341e-01,-8.205193e-02,-7.784407e-01,-6.540392e-01,-8.016231e-01,-6.016673e-01,-5.273046e-01,-5.220312e-01,-1.836174e-01,-4.321561e-01,-6.463637e-01,-2.571139e-01,-1.232860e+00,-2.733414e-01,-6.633573e-01,-6.240652e-01,1924.000000
50%,-2.709587e-01,1.017507e-01,6.751141e-02,7.816830e-02,-3.464320e-02,5.402006e-02,3.495959e-01,4.446455e-02,1.208054e-01,1.211515e-01,-3.808313e-01,2.484375e-01,-7.053192e-02,-1.209864e-01,-1.603836e-01,-1.057245e-01,-2.531977e-01,-2.236935e-01,2.210613e-01,9.055634e-02,-1.204053e-01,1.863660e-01,2.255994e-01,-4.171101e-02,-1.209619e-01,-3.104091e-01,2227.000000
75%,7.878483e-01,7.560007e-01,6.840443e-01,7.635037e-01,6.898359e-01,7.476849e-01,6.308840e-01,9.185786e-01,7.553051e-01,5.109511e-01,8.736757e-01,5.507997e-01,7.468685e-01,5.234879e-01,7.785920e-01,4.887968e-01,6.168175e-01,6.499852e-01,5.970260e-01,4.371478e-01,4.477769e-01,6.267493e-01,8.112377e-01,3.329088e-01,3.480433e-01,2.588798e-01,2459.000000
max,2.873213e+00,2.887961e+00,2.958514e+00,3.359160e+00,3.133894e+00,2.575124e+00,1.746400e+00,2.064750e+00,2.842186e+00,1.336407e+01,2.314012e+00,2.875327e+00,2.984749e+00,3.204730e+00,2.757874e+00,2.342625e+00,2.737697e+00,2.697149e+00,1.733416e+00,7.118025e+00,3.441431e+00,3.025287e+00,2.813213e+00,2.952091e+00,3.958997e+00,4.179112e+00,3783.000000


## 2) Run the RBA Model in BQML

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = '' #add proj name and dataset

In [ ]:
'''
Create a linear model to measure the impact of digital media (x variables) on conversions (y variable).
Hyperparameter tuning can be added into the options section of the query
'''

%%bigquery
CREATE OR REPLACE MODEL `.RBA_demo.RBA_model1`  #update project name
OPTIONS (model_type='linear_reg',
         input_label_cols = ['y1'])
AS SELECT x8, x9, x10, x11, x12, x14, x16, x26, x27, x28, x29, x30, x32, x33, 
x34,x35, x37, x38, x39, x40, x41, x42, x43, x44, x45, x46, y1
    FROM `.RBA_demo.cleaned_data`; #update project name

""


### 2.1) Print the model coefficient results

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = '' #add proj name and dataset

In [ ]:
'''
Call the model coefficient weights from the model and save to a dataframe "model_coefficients_results".
The standardize parameter is an optional parameter that determines whether the model 
weights should be standardized to assume that all features have a mean of zero and a 
standard deviation of one. Standardizing the weights allows the absolute magnitude 
of the weights to be compared to each other.
'''

%%bigquery model_coefficients_results
SELECT
  *
FROM
  ML.WEIGHTS(MODEL `.RBA_demo.RBA_model1`,
    STRUCT(true AS standardize)) #update project name

In [ ]:
model_coefficients_results

### 2.2) Print the model evaluation metrics

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = '' #add proj name and dataset

In [ ]:
'''
Call the model evaluation metrics from the model and save to a dataframe "evaluation_metrics".
For linear regression models The ML.EVALUATE function returns: mean absolute error,
mean squared erorr, mean squared log error, median absolute error, r-squared, and
explained variance metrics.
'''

%%bigquery evaluation_metrics
SELECT *
FROM ML.EVALUATE(MODEL `db.dataset.rba_model`) #update project name

## 3) Calculate contribution of each digital media tactic on conversions

In [ ]:
'''
Use the relativeImp package (https://pypi.org/project/relativeImp/)
Conducts key driver analysis to generate relative importance by feature in the model

The relativeImp function produces a raw relative importance and a normalized relative 
importance value. Raw relative importance sums to the r-squared of the linear model.
Normalized relative importance is scaled to sum to 1
'''

conversions = 'y1'
tactics = ['x2','x16','x17','x26','x27','x32','x34','x35','x38','x39','x45']
relative_importance_results = relativeImp(df, 
                                          outcomeName = conversions, 
                                          driverNames = tactics)

In [ ]:
relative_importance_results

## 4) Validate Linear Regression Model Assumptions

In [ ]:
'''
For any statistical model it is important to validate model assumptions.
With RBA, we validate the standard linear model assumptions of:
  - Linearity
  - Normality of errors
  - Absence of multicollinearity
  - Homoscedasticity
  - Absence of autocorrelation of residuals


If any of the model assumptions fail, a different model specification, as well
as re-examination of the data should be considered

Incorrect model use can lead to unreliable results
'''

### 4.1) Generate model predictions and residuals

In [ ]:
################################################################################
######################### CHANGE BQ PROJECT NAME BELOW #########################
################################################################################

project_name = '' #add proj name and dataset

In [ ]:
'''
Select the predicted conversions (y1) of the model and actual conversions from the data (y1)
using the ML.PREDICT function
'''

%%bigquery model_predictions
SELECT
  predicted_y1, y1
FROM
  ML.PREDICT(MODEL `db.dataset.rba_model`, #update project name
    (
    SELECT
        *
    FROM
      `db.dataset.rba_input_table`)); #update project name

In [ ]:
'''
Calculate model residuals as the difference from predicted y1 values and actual
y1 values
'''
model_predictions['residuals'] = model_predictions.predicted_y1 - model_predictions.y1

### 4.2) Linearity

In [ ]:
'''
Visually inspect linearity between target variable (y1) and predictions
'''
plt.plot(model_predictions.predicted_y1,model_predictions.y1,'o',alpha=0.5)
plt.show()

### 4.3) Normality of Errors

In [ ]:
'''
Visually inspect the residuals to confirm normality
'''

fig = sm.qqplot(model_predictions.residuals)
sns.kdeplot(model_predictions.residuals, label = '', shade = True)
plt.xlabel('Model Residuals'); plt.ylabel('Density'); plt.title('Distribution of Residuals');

### 4.4) Absence of Multicollinearity

In [ ]:
'''
Multicollinearity was checked and handled during data pre-processing stage.
'''

### 4.5) Homoscedasticity


In [ ]:
'''
Visually inspect residuals to confirm constant variance
'''
plt.plot(model_predictions.residuals,'o',alpha=0.5)
plt.show()

### 4.6) Absence of Autocorrelation of the residuals

In [ ]:
'''
The Durbin Watson test is a statistical test for detecting autocorrelation of the 
model residuals
'''

dw = durbin_watson(model_predictions.residuals)
print('Durbin-Watson',dw)

In [ ]:
if dw < 1.5:
        print('Positive autocorrelation', '\n')
elif dw > 2.5:
        print('Negative autocorrelation', '\n')
else:
        print('Little to no autocorrelation', '\n')
